# Hyperparameter Tuning

All machine learning models contain hyperparameters than you can tune (as if they were knobs on a guitar) to alter how the learning happens. By changing the hyperparameters, the model can be specified at a different setting. Different datasets require different hyperparameter settings for optimal performance. Each class of machine learning models has different hyperparameters. For instance, a [decision tree classifier][1] has many hyperparameters, some of which are:

* **max_depth** : The maximum depth of the tree. Default is to continue making splits until node is "pure" (All target values are the same)
* **min_samples_split** : The minimum number of samples required to split an internal node. Default is 2
* **min_samples_leaf** : The minimum number of samples required to be in a leaf node. Default is 1

[1]: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

## Hyperparameters vs Parameters

Hyperparameters are all the knobs that you have control over. You set them during **instantiation**. They do NOT change during training. Many of these models have "normal" parameters that are learned during **training** when calling the `fit` method. These parameters are completely different than hyperparameters. The algorithm will find the optimal value of these parameters during training based on the evaluation metric that it uses. For instance, in simple linear regression, the parameters are the slope and intercept.

Hyperparameters don't change after you have instantiated your model. In a decision tree, a hyperparameter such as `max_depth` dictates the maximum depth of the tree. The algorithm must stop growing the tree when it reaches that depth. The location of where and how the splits take place is learned through the model. These decisions are what is learned and can be thought of as a normal parameter for a decision tree.

Technically, you can change hyperparameters after instantiating your model, but these changes are completed by you. They are not learned. This is like purchasing a car and then choosing to paint it a different color.

## Overfitting

Previously, we saw a decision tree severely overfit the data by having a test score significantly worse than a training score. Let's recreate that example below:

In [ ]:
import pandas as pd
housing = pd.read_csv('../data/housing_sample.csv')
X = housing[['GrLivArea', 'GarageArea', 'FullBath']]
y = housing['SalePrice']
X.head()

Let's fit our decision tree on all of our data and evaluate it improperly on the data that it was trained on. This results as nearly a perfect score.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X, y)
dtr.score(X, y)

As we just learned, proper evaluation is done with cross validation. Here, we use five folds, shuffle the data, and report the mean test score.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
cross_val_score(dtr, X, y, cv=kf).mean()

When the cross-validated scores are much worse than the training score, it's likely that your model is **overfitting** to the training data. The model is learning too much about the particulars of the data that it trains on. The model isn't able to adapt to any of the nuances in the new dataset.

### Decision trees are prone to overfitting

The decision tree model is extremely flexible and has the ability to overfit to the training set more than almost any other type of model. It's able to do this because scikit-learn allows the tree to grow until there is just a single observation left in each node. This means that the tree has the ability to map each input to exactly one output.

### How does a decision tree map each input to one output

A decision tree can use the same feature to make any number of decisions. Let's use the first observation as an example which has a ground living area of 1,710, garage area of 548 and 2 full baths. The very fist condition could test whether ground living area less than 2,000. The second could test whether ground living area is less than 1,700. The third could test whether ground living area is less than 1,715. The conditions can continually narrow the range of the possible values for each feature until each unique input is a leaf on its own. 

If there are multiple observations that share the exact same inputs, then it won't be possible to map each input to a single output. This is what occurred with our data above. We can use some grouping in pandas to determine the duplicated values in the dataset and the frequency of occurrence. Houses with the exact same values for these three features cannot be split further.

In [ ]:
X.groupby(['GrLivArea', 'GarageArea', 'FullBath']).size() \
 .sort_values(ascending=False).reset_index(name='num_duplicated').head()

### Memorization

Continually making decisions until no more splits are possible is no different than memorizing the output for each input. You will have essentially built a database, albeit a very inefficient one. Just like memorizing test answers won't help you on an actual test, memorizing inputs and their corresponding output isn't going to produce good performance in the future.

## Inspecting the decision tree

Let's inspect the tree to find out some of its properties.

In [ ]:
tree = dtr.tree_

In [ ]:
from sklearn.tree import plot_tree

The `max_depth` attribute contains the maximum depth of the tree. Not all decision paths are this long as certain paths can be much shorter due to reaching the stopping criteria earlier.

In [ ]:
tree.max_depth

The tree makes up to 23 decisions in order to reach a leaf. This number of decisions is likely way too many. The total number of nodes and leaves are stored in the `node_count` and `n_leaves` attribute respectively.

In [ ]:
tree.node_count

In [ ]:
tree.n_leaves

This is an enormous tree and at this size would not be very interpretable. Do not attempt to plot this tree as creating nearly 3,000 nodes will crash your Jupyter Notebook.

## Change hyperparameters to reduce overfitting

We can reduce the amount of overfitting by specifying different hyperparameters during model instantiation. One of the best hyperparameters to set for a decision tree is `max_depth`. This limits the depth of the tree to the integer you provide it. 

By default, this parameter is set to `None`, which from the documentation, states that the tree will grow until 'all leaves are pure or until all leaves contain less than `min_samples_split` samples'. Leaves are considered **pure** whenever all the y-values for the observations are the same. 

An example can make this clearer. If a current node contains 300 houses all having less than 1,500 square feet of ground living area, more than 500 square feet of garage area and 2 full baths all priced at 150,000 dollars, then it would be considered pure. There would be nothing to be gained by splitting the data further since the sale price of each house is the same. The feature values for each house could all be different, but as long as the y-values are all the same the node is pure.

The tree will also stop growing if the number of samples is less than `min_samples_split`, which is another hyperparameter and set to 2 by default. Remember that the term samples also refers to observations. The only number less than 2 is 1, so the default value will do nothing to prevent the tree from growing.

There's one other instance where the tree will stop growing and that is when all the features are the exact same, which happened in a few cases as we reported above.

### Set the `max_depth` hyperparameter

Now that we have more insight into the `max_depth` hyperparameter, let's set it to 8 when we instantiate a new decision tree and calculate a new training score for it.

In [ ]:
dtr = DecisionTreeRegressor(max_depth=8)
dtr.fit(X, y)
dtr.score(X, y)

The tree cannot grow to a depth greater than 8 and therefore cannot memorize the data as it did with the original tree. The score is no longer near perfect. Let's get the number of nodes and leaves of our new tree.

In [ ]:
tree = dtr.tree_
tree.node_count

In [ ]:
tree.n_leaves

The tree is nearly one-tenth of the size as the original though it is still quite large and would be difficult to interpret and nearly impossible to visualize on any reasonably sized sheet of paper.

### Cross validate with new hyperparameter

We are more interested in the cross validation score and not the training score. Let's find it now.

In [ ]:
cross_val_score(dtr, X, y, cv=kf).mean()

This is a significant improvement over the original score and we would expect this new model using the `max_depth` of 8 to perform significantly better in the future than the original. Let's now set `max_depth` to 1, which will produce a very short tree with only one split and two leaves. Let's get the training and cross-validated scores.

In [ ]:
dtr = DecisionTreeRegressor(max_depth=1)
dtr.fit(X, y)
dtr.score(X, y)

In [ ]:
cross_val_score(dtr, X, y, cv=kf).mean()

This is the lowest training score we've encountered for a decision tree on this data. Limited to just one split, the decision tree just can't learn enough to make good predictions. When our model learns too little, it is **underfitting**.

### Iterate to find the best value for `max_depth`

The value 8 for `max_depth` was picked arbitrarily. There is no reason to believe that it is the best choice. To help find a better choice, we can iterate over many candidate values of `max_depth` to find the best one. The function `get_scores` is created to automate the procedure for any hyperparameter. Pass it the name of the hyperparameter, all the hyperparameter values you want to test, and the regressor class. It stores all the cross-validated scores in a DataFrame and plots the scores for each hyperparameter value.

In [ ]:
def get_scores(hp_name, hp_values, Regressor):
    data = {hp_name: [], 'score': []}
    for value in hp_values:
        reg = Regressor(**{hp_name: value})
        score = cross_val_score(reg, X, y, cv=kf).mean().round(3)
        data['score'].append(score)
        data[hp_name].append(value)
    df_scores = pd.DataFrame(data)
    df_scores.plot(x=hp_name, y='score', kind='line', figsize=(10, 4));
    return df_scores

The mean cross validated scores are plotted for all integers 1 through 20 for the hyperparameter `max_depth`.

In [ ]:
df_scores = get_scores('max_depth', range(1, 21), DecisionTreeRegressor)

Let's output some of the actual scores where the best value for `max_depth` is found.

In [ ]:
df_scores.query('3 <= max_depth <= 8')

## Model selection

The name of this part of the book is titled 'Model Selection'. We ultimately want to select a model that has the best combination of hyperparameters that will give us the best performance on unseen data in the future. Our work above helped us find a good choice for the `max_depth` hyperparameter.

It appears that the optimal `max_depth` is between 3 and 7. While this is great information, we don't have a final model to use to make predictions on future data. To obtain this final model, we choose a single `max_depth` and fit it on all the data. Here, we use 4, though any of the other values with a similar score can work as well. Getting a slightly better cross-validated score doesn't mean that the actual future performance is guaranteed to be better using that value.

In [ ]:
dtr_final = DecisionTreeRegressor(max_depth=4)
dtr_final.fit(X, y);

The `dtr_final` decision tree can now be used to make predictions on future data. It is the model we have selected. We expect to have an $R^2$ of around .58. 

## Optimizing other hyperparameters

Decision trees have several hyperparameters that are worth tuning to build a better model. We cover a few of them below.

### `min_samples_split`

Another common hyperparameter to tune for decision trees is `min_samples_split` which is also used to prevent the tree from growing too large. A tree stops growing a particular branch if the number of samples in the node is less than `min_samples_split`. For example, if we set `min_samples_split` to 30 and there are 29 samples in the current node, then no more splits will take place. As mentioned, the default value is 2, which won't do anything to prevent the tree from growing. 

Let's instantiate a new decision tree using 100 as the value for `min_samples_split` and fit it to the data. We will not set a value for `max_depth` with this example so it will be set to its default value `None` meaning that the tree can grow arbitrarily deep as long as the node contains more than 100 samples.

In [ ]:
dtr = DecisionTreeRegressor(min_samples_split=100)
dtr.fit(X, y)
dtr.score(X, y)

Our training score is lower than the tree which was built with a `max_depth` of 8. Let's inspect the attributes of the tree.

In [ ]:
tree = dtr.tree_
tree.max_depth

In [ ]:
tree.node_count

In [ ]:
tree.n_leaves

Interestingly, the `max_depth` of the tree is 10, which is larger than the tree fit with a `max_depth` of 8, but the number of nodes and leaves are significantly less. This implies that many of the branches were stopped well short of the maximum depth of the tree and perhaps only a few were responsible for the maximum depth. Let's properly evaluate it using cross validation.

In [ ]:
cross_val_score(dtr, X, y, cv=kf).mean()

Let's iterate through many possible values of `min_samples_split` to attempt to find the best one and then plot the results.

In [ ]:
min_samples_splits = [2, 5, 10, 20, 50, 100, 200, 300, 500]
df_scores = get_scores('min_samples_split', min_samples_splits, DecisionTreeRegressor)

The best value seems to be at 50. Surprisingly a very high value like 500 (which is around 1/3 the total size of the data) still does a decent job. The low values do poorly as they do little to prevent a deep decision tree from growing. Let's output the scores from greatest to least.

In [ ]:
df_scores.sort_values('score', ascending=False)

### `min_samples_leaf`

One issue with `min_samples_split` is that it doesn't prevent a leaf from having as few as a single sample in it. For example, a tree with `min_samples_split` set at 100 could have a node with 150 samples in it and split it such that one node has 148 samples and the other has 2. Having a leaf with such a small number of observations may not be desirable.

The `min_samples_leaf` hyperparameter sets the minimum number of samples that must be present in the left and right child nodes in order for the split to take place. If the number of samples is below the value of `min_samples_leaf`, then that branch ceases to grow. Let's choose some values and plot the cross-validated scores using our `get_scores` function.

In [ ]:
min_samples_leafs = [2, 5, 10, 15, 25, 50, 100, 200]
df_scores = get_scores('min_samples_leaf', min_samples_leafs, DecisionTreeRegressor)

In [ ]:
df_scores.sort_values('score', ascending=False)

The best value appears to be somewhere between 15 and 25. The scores are slightly higher than they were for `min_samples_split`.

## Setting multiple hyperparameters simultaneously

All of the hyperparameters for each model have a default value provided by scikit-learn. Even if you do not set a hyperparameter directly, it will get set to something. All estimators have a `get_params` method that returns a dictionary of the parameter name and its corresponding value. Let's instantiate a new decision tree and call this method to see the default hyperparameter values.

In [ ]:
dtr = DecisionTreeRegressor()
dtr.get_params()

Thus far, we have only set one hyperparameter at a time. It's possible for us to set any number of hyperparameters simultaneously to customize the construction of our model. For example, let's instantiate a decision tree so that it has a `max_depth` of 8 and a `min_samples_leaf` of 50 and then verify that these were set correctly with the `get_params` method.

In [ ]:
dtr = DecisionTreeRegressor(max_depth=8, min_samples_leaf=50)
dtr.get_params()

Evaluating our model using cross validation returns the following score.

In [ ]:
cross_val_score(dtr, X, y, cv=kf).mean()

### Iterate over many combinations

A new function, `get_scores2` is defined such that it iterates over all the given combinations of two hyperparameter settings, performs cross validation and returns a DataFrame that colors each cell a color proportional to its score. The higher the score, the redder the cell.

In [ ]:
def get_scores2(hp_name1, hp_values1, hp_name2, hp_values2, Regressor):
    data = {hp_name1: [], hp_name2: [], 'score':[]}
    for val1 in hp_values1:
        for val2 in hp_values2:
            reg = Regressor(**{hp_name1: val1, hp_name2: val2})
            score = cross_val_score(reg, X, y, cv=kf).mean().round(3)
            data['score'].append(score)
            data[hp_name1].append(val1)
            data[hp_name2].append(val2)
    df = pd.DataFrame(data).pivot(index=hp_name1, columns=hp_name2, values='score')
    return df.style.background_gradient('coolwarm', axis=None)

Let's search multiple combinations of `max_depth` and `min_samples_leaf`.

In [ ]:
get_scores2('max_depth', range(2,10), 'min_samples_leaf', [2, 5, 10, 20, 50, 100], 
            DecisionTreeRegressor)

The best combination appears to be a `max_depth` of 6 and a `min_samples_leaf` of 20. This combination did slightly better than just using a `min_samples_leaf` of 15 by itself. To obtain a final model, we need to train all of the data with this combination.

In [ ]:
dtr_final = DecisionTreeRegressor(max_depth=6, min_samples_leaf=20)
dtr_final.fit(X, y);

We can continue to iterate over even more combinations of hyperparameters to find the very best set. Manually writing functions like this is tedious. Thankfully, scikit-learn has a meta-estimator called `GridSearchCV` that automates this procedure for us. We will learn about in an upcoming chapter.

###  Why are the hyperparameter defaults for decision trees so poor?

The default values for the hyperparameters of decision trees are not good and you will almost always change them. Allowing a tree to grow until it has memorized the data as the default seems like a poor choice. I'm not sure what the alternative is, but scikit-learn certainly expects you to tune the decision tree yourself.

## Hyperparameter tuning with k-nearest neighbors

All models have hyperparameters that can be tuned to get better performance. Decision trees are an example of a model that contain many hyperparameters that are set to poor default values. We now turn to the k-nearest neighbors model which has fewer hyperparameters. They are set to reasonable default values giving them better decent performance without having to tune them at all. Regardless, it's still important to tune k-nearest neighbors as you can get significantly better models than the defaults.

Let's begin by instantiating a k-nearest neighbors estimator and viewing the default hyperparameter values with the `get_params` method.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor()
knr.get_params()

### The number of neighbors

The `n_neighbors` hyperparameter controls the number of neighbors and is the most crucial one to optimize. Let's use our `get_scores` function from above to help us find the optimal number of neighbors.

In [ ]:
neighbors = [1, 2, 3, 4, 5, 6, 7, 10, 15, 20, 40, 60, 100]
df_scores = get_scores('n_neighbors', neighbors, KNeighborsRegressor)

The best choice for neighbors appears to be between 10 and 40.

In [ ]:
df_scores.sort_values('score', ascending=False).head()

The larger the number of neighbors, the more computational time it will take. If multiple choices for number of neighbors produce a similar score, you might want to choose the smallest number. Here, we choose 10 neighbors for our final model.

In [ ]:
knr_final = KNeighborsRegressor(n_neighbors=10)
knr_final.fit(X, y);

The `weights` hyperparameter is worth tuning. The default value is 'uniform' which treats all neighbors equally when making predictions. It may be set to the string 'distance' to weigh the predictions proportional to the distance between the neighbor and the predicted point. Let's use `get_scores2` to find the best combination of values for hyperparameters `n_neighbors` and `weights`. 

In [ ]:
get_scores2('weights', ['uniform', 'distance'], 'n_neighbors', neighbors, KNeighborsRegressor)

## Hyperparameter tuning linear regression

Our current specification for the linear regression model does not have many hyperparameters to tune. Let's import it into our namespace and get the default hyperparameter values with the `get_params` method.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.get_params()

Both `copy_X` and `n_jobs` aren't actual hyperparameters and won't affect the model outcome. The two actual hyperparameters are `fit_intercept` and `normalize`. By default, scikit-learn will fit an intercept for linear regression, which is the predicted value when all features are 0. By default, this value is `True`. Setting it to `False` will force the intercept to be 0. Unless you have a special requirement, fitting an intercept is almost always desired.

The `normalize` parameter normalizes the data which scales the data. Scaling will be discussed in the Data Transformations part of the book. Fitting the intercept gives us just a slightly better model.

In [ ]:
get_scores2('fit_intercept', [True, False], 'normalize', [True, False], LinearRegression)

## Setting hyperparameters is like setting specifications for a car

In this section, I am going to make an analogy between machine learning estimators and cars. The uninstantiated estimator will be analogized to a blueprint of a car and the hyperparameters will be analogized to different features of a car.

Before instantiation, the machine learning estimators that we import are like blueprints. They are instructions on how to create a 'machine learning vehicle' that learns from data. We cannot learn from data with the blueprint since it is merely a sheet of instructions at this point. The instructions of the blueprint can be used to create any number of machine learning vehicles. In Python, we construct a new machine learning vehicle by instantiating our estimator. Up until this chapter, we have been instantiating our models with the default values of the hyperparameters.

Let's make the connection to cars and pretend we have a blueprint for a Toyota Prius. Like the machine learning estimator, the blueprint is not the physical car, but the instructions on how to construct the car. This blueprint may be used to build any number of cars. Like machine learning models, there are many different specifications that may be set for your car. For instance, these specifications could be the car color, wheel size, number of horsepower, type of dashboard display, type of transmission, and dozens more. Like machine learning estimators, these specifications have default values.

When a car is built, the specifications can be set to build different cars for different purposes. Perhaps we want more horsepower to give us more acceleration or want four-wheel drive to give us more power off-road. There are many reasons to build cars with different specifications than their defaults. Machine learning estimators are quite similar. We tune the hyperparameters for different needs.

## Exercises

### Exercise 1
<span  style="color:green; font-size:16px">Search different combinations of hyperparameters for a random forest. Add more features to your model and search `max_features` and `n_estimators` as well as others that are in-common with decision trees.</span>